# 👕 LangGraph: Clothing Store Competitor Insights Generator
This notebook implements a multi-node LangGraph pipeline to fetch competitor store data and generate analysis reports.

In [ ]:
# 📦 Install Dependencies
!pip install langgraph langchain openai python-dotenv requests

In [ ]:
# 🔐 Load Environment Variables
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

In [ ]:
# 🧠 Define LangGraph Nodes
from langgraph.graph import StateGraph, END
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
import requests

# Initialize LLM
llm = ChatOpenAI(model="gpt-4", openai_api_key=OPENAI_API_KEY)

# Node 1: Search competitors
def search_competitors(state):
    location = state['location']
    headers = {"X-API-KEY": SERPER_API_KEY, "Content-Type": "application/json"}
    payload = {
        "q": f"clothing stores in {location}",
        "gl": "in",
        "hl": "en"
    }
    response = requests.post("https://google.serper.dev/places", json=payload, headers=headers)
    response.raise_for_status()
    state['search_results'] = response.json()['places']
    return state

# Node 2: Generate report
def generate_llm_report(state):
    location = state['location']
    search_results = state['search_results']
    context = "\n".join([
        f"{place['title']} at {place['address']} with rating {place.get('rating', 'N/A')}" for place in search_results
    ])
    prompt = f"""
You are a retail business analyst. Based on the following data on clothing stores in {location}, generate a detailed competitor insights report.

Data:
{context}

Instructions:
1. Identify the top 3 competitors based on ratings or reviews.
2. Estimate their likely peak footfall hours based on address and local behavior.
3. Suggest strategic actions for a new clothing store to compete effectively.

Generate this in a readable bullet-point format.
"""
    response = llm([HumanMessage(content=prompt)])
    state['report'] = response.content
    return state

In [ ]:
# 🧩 Build and Run the LangGraph
# Create the graph
graph = StateGraph()
graph.add_node("search", search_competitors)
graph.add_node("report", generate_llm_report)
graph.set_entry_point("search")
graph.add_edge("search", "report")
graph.set_finish_point("report")

app = graph.compile()

# Run it
initial_state = {'location': 'Koramangala, Bangalore'}
final_state = app.invoke(initial_state)
print("\n📊 Report Generated by LLM:\n")
print(final_state['report'])